## Неделя 2. Понедельник
### Обучение с учителем

### Применение базовых методов классификации

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
import joblib
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import joblib
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.model_selection import cross_val_score


#### 0. Ознакомьтесь с датасетом

In [23]:
df = pd.read_csv('heart.csv')
print(df.head())
print(df.info())
print(df.isnull().sum())

    Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0  40.0   M           ATA        140          289          0     Normal   
1  49.0   F           NAP        160          180          0     Normal   
2  37.0   M           ATA        130          283          0         ST   
3  48.0   F           ASY        138          214          0     Normal   
4  54.0   M           NAP        150          195          0     Normal   

   MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0    172              N      0.0       Up             0  
1    156              N      1.0     Flat             1  
2     98              N      0.0       Up             0  
3    108              Y      1.5     Flat             1  
4    122              N      0.0       Up             0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   A

* __Age__: age of the patient [years]
* __Sex__: sex of the patient [M: Male, F: Female]
* __ChestPainType__: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
* __RestingBP__: resting blood pressure [mm Hg]
* __Cholesterol__: serum cholesterol [mm/dl]
* __FastingBS__: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
* __RestingECG__: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite * left ventricular hypertrophy by Estes' criteria]
* __MaxHR__: maximum heart rate achieved [Numeric value between 60 and 202]
* __ExerciseAngina__: exercise-induced angina [Y: Yes, N: No]
* __Oldpeak__: oldpeak = ST [Numeric value measured in depression]
* __ST_Slope__: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
* __HeartDisease__: output class [1: heart disease, 0: Normal]

* Таргетом является столбец `HeartDisease`. Необходимо предсказать по имеющимся данным, есть ли проблемы с сердцем

#### 1. Небольшие рекомендации ниже 


* __Baseline pipeline (базовый пайплайн)__ - это простой пайплайн, который используется как отправная точка или точка сравнения при разработке и оценке более сложных моделей или алгоритмов. 

* Для этого сначала используйте самые простые идеи по заполнению пропусков(средними, медианами, модами) и кодированию категориальных данных, которые вам приходят в голову. 

* После того, как вы построите модели провалидируете их. Можно будет приступать к попыткам улучшить свою модель с помощью ваших идей - пробовать создавать новые фичи, кодировать данные по-другому, заполнять иначе NaN и тд

#### 2. Заполните пропущенные значения(`Imputing`), как считаете нужным.  

- Не забывайте памятку выше, сначала заполняйте самыми тривиальными идеями. Например, средними, медианами и т.д

In [24]:
numeric_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
categorical_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

numeric_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

my_imputer = ColumnTransformer([
    ('num_imputer', numeric_imputer, numeric_features),
    ('cat_imputer', categorical_imputer, categorical_features)
], remainder='passthrough')


##### 2.1 Оберните в `ColumnTransformer` свой `Imputing` данных. Проверьте корректность его работы. Для этого необходимо сделать:

1. Обучить и трансформировать свой `Imputer` с помощью `your_imputer.fit_transform` - на тренировочных данных
2. Заполнить с помощью `your_imputer.transform` - на тестовых данных

Убедитесь, что данные прошли через этап `Imputing'а` и пропусков в них больше нет

In [25]:
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_imputed = my_imputer.fit_transform(X_train)
X_valid_imputed = my_imputer.transform(X_valid)


#### 3. Закодируйте категориальные переменные, как считаете нужным

* `OneHotEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)  
* `TargetEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html)  
* `OrdinalEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)  
* `CatBoostEncoding` (https://www.geeksforgeeks.org/categorical-encoding-with-catboost-encoder/)  

In [26]:
my_encoder = ColumnTransformer([
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'), categorical_features)
], remainder='passthrough')


##### 3.1 Оберните в `ColumnTransformer` свой `Encoding` данных. Проверьте корректность его работы. 

In [27]:
X_train_encoded = my_encoder.fit_transform(X_train)
X_valid_encoded = my_encoder.transform(X_valid)

#### 4. То же самое проделать с нормализацией данных

* `StandardScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* `MinMaxScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
* `RobustScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)

In [28]:
my_scaler = ColumnTransformer([
    ('num_scaler', StandardScaler(), numeric_features)
], remainder='passthrough')

#### 4.1 Оберните в `ColumnTransformer` свой `Scaling` данных, проверьте корректность работы.

In [29]:
X_train_scaled = my_scaler.fit_transform(X_train)
X_valid_scaled = my_scaler.transform(X_valid)

#### 5. Соберите весь препроцессинг в общий Pipeline.

In [30]:
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_features),

    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features)
], remainder='passthrough')


##### 5.1 Прогоните свои данные через `preprocessor` и убедитесь, что ваши данные проходят через него корректно и уже готовы к ML-модели

In [31]:
preprocessor.fit(X_train)
X_train_ready = preprocessor.transform(X_train)
X_valid_ready = preprocessor.transform(X_valid)


#### 6.ML-модели

* `LogisticRegression` (из `sklearn.linear_model`)  
* `LogisticRegression with regularization` (из `sklearn.linear_model`)  
* `KNeighborsClassifier` (из `sklearn.neighbors`)  
* `DecisionTree` (из `sklearn.tree`)  

##### 6.1 Обучите свой `Pipeline` с помощью метода `.fit()` с разными моделями.

In [32]:
models = {
    'LogReg': LogisticRegression(max_iter=1000),
    'LogReg with l1': LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000),
    'LogReg with l2': LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Tree': DecisionTreeClassifier(random_state=42)
}


#### 7. С помощью метода `.predict()` (на вход поступают только матрица признаков, без целевой переменной) предсказать значения на обучающей выборке (`X_train`) и валидационной выборке (`X_valid`).

In [33]:
models = {
    'LogReg': LogisticRegression(max_iter=1000),
    'LogReg with l1': LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000),
    'LogReg with l2': LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Tree': DecisionTreeClassifier(random_state=42)
}

predictions = {}

for name, model in models.items():
    ml_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    ml_pipeline.fit(X_train, y_train)
    
    predictions[name] = {
        'train': ml_pipeline.predict(X_train),
        'valid': ml_pipeline.predict(X_valid)
    }


##### 7.1 С помощью функции оценки качества (`accuracy_score`) собрать следующую таблицу ниже

* значение функции на обучающих данных
* значение функции на валидационных данных 
    
Результатом выполнения этого пункта будет `DataFrame` формата: 
    
|  |train|valid|
|--|-----|-----|
|**LogReg**|  train_score  | valid_score    |
|**LogReg with l1**|  train_score  | valid_score    |
|**LogReg with l2**|  train_score  | valid_score    |
|**KNN**| train_score  |  valid_score   |
|**Tree**| train_score | valid_score    |

In [34]:
results = {}

for name, preds in predictions.items():
    train_acc = accuracy_score(y_train, preds['train'])
    valid_acc = accuracy_score(y_valid, preds['valid'])
    results[name] = {'train': train_acc, 'valid': valid_acc}

import pandas as pd

results_df = pd.DataFrame(results).T
results_df.index.name = ''
results_df.columns = ['train', 'valid']
results_df


,train,valid
,,
LogReg,0.870572,0.853261
LogReg with l1,0.870572,0.853261
LogReg with l2,0.871935,0.853261
KNN,0.889646,0.869565
Tree,1.000000,0.815217


#### 8. Теперь реализуйте __кросс-валидацию__ с KFold=5 и выведите средний __score__

In [35]:
models_cv = {
    'LogReg': LogisticRegression(max_iter=1000),
    'LogReg with l1': LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000),
    'LogReg with l2': LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'SVC': SVC(),
    'Tree': DecisionTreeClassifier(random_state=42)
}

cv_results = {}

for name, model in models_cv.items():
    ml_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(ml_pipeline, X, y, cv=5, scoring='accuracy')
    cv_results[name] = np.mean(scores)

cv_df = pd.DataFrame.from_dict(cv_results, orient='index', columns=['cross_val_score'])
cv_df


,cross_val_score
LogReg,0.828938
LogReg with l1,0.834385
LogReg with l2,0.828938
KNN,0.838721
SVC,0.832163
Tree,0.746086


|  |cross_val_score|
|--|-----|
|**LogReg**|  0.828938 |
|**LogReg with l1**|  0.834385  |
|**LogReg with l2**|  0.828938  |
|**KNN**| 0.838721  |
|**SVC**| 0.832163  |
|**Tree**| 0.746086 |

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Пора сохранить изменения для __github__. 

1. Перейди в командной строке в папку, в которой расположен этот нотбук. 
2. Выполни команду `git add 06-01-task.ipynb`
3. Выполни команду `git commit -m "base models in progress"`
4. Выполни команду `git push`

##### 9. Теперь, когда вы проделали весь pipeline и обучили базовую модель, можно вернуться к началу и пробовать новые идеи и искать точки роста для ваших моделей, в том числе и добавление новых фичей

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__ и выполни команду `!git status` в ячейке ниже.


## Неделя 2. Вторник 
### Обучение с учителем
### Применение ансаблей моделей для задач классификации и регрессии

1. Загружаем предобработанный датасет (либо загружаем и очищаем, если не осталось сохраненной версии)
2. К прежним датасетам применяем ансамбли моделей: 

    - `RandomForest`
    - `Voting`
    - `Catboost`([https://catboost.ai/en/docs/concepts/python-installation](https://catboost.ai/en/docs/concepts/python-installation))
    - `LightGBM`([https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html))
    - `XGBoost` ([https://xgboost.readthedocs.io/en/latest/install.html](https://xgboost.readthedocs.io/en/latest/install.html))
# ❓

In [36]:
ensemble_models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    "CatBoost": CatBoostClassifier(n_estimators=100, max_depth=10, random_seed=42, verbose=0),
    "LightGBM": LGBMClassifier(n_estimators=100, max_depth=10, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=100, max_depth=10, random_state=42)
}

ensemble_results = {}
for name, model in ensemble_models.items():
    pipe = Pipeline([('preprocessor', preprocessor), ('model', model)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_valid)
    ensemble_results[name] = accuracy_score(y_valid, y_pred)

ensemble_df = pd.DataFrame.from_dict(ensemble_results, orient='index', columns=['accuracy'])
print(ensemble_df.sort_values('accuracy', ascending=False))

best_models = [
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)),
    ('cat', CatBoostClassifier(n_estimators=100, max_depth=10, random_seed=42, verbose=0)),
    ('lgb', LGBMClassifier(n_estimators=100, max_depth=10, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=100, max_depth=10, random_state=42))
]

voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_pipe = Pipeline([('preprocessor', preprocessor), ('classifier', voting_clf)])
voting_pipe.fit(X_train, y_train)
voting_acc = accuracy_score(y_valid, voting_pipe.predict(X_valid))
print(f"Voting Classifier Accuracy: {voting_acc:.4f}")

ensemble_results['Voting'] = voting_acc
ensemble_df = pd.DataFrame.from_dict(ensemble_results, orient='index', 
                                   columns=['accuracy']).sort_values('accuracy', ascending=False)
print("\nИтоговые результаты ансамблей:")
print(ensemble_df)

[LightGBM] [Info] Number of positive: 401, number of negative: 333
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 375
[LightGBM] [Info] Number of data points in the train set: 734, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546322 -> initscore=0.185819
[LightGBM] [Info] Start training from score 0.185819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

### Какие значения по умолчанию заданы в случайном лесе для числа деревьев и их глубины?

По умолчанию:
n_estimators=100 — число деревьев
max_depth=None — глубина не ограничена (дерево строится до максимального разделения)


3. Прогоните модели через `optuna`, определяем лучшие параметры и сохраняем результат в словарь вида:
python
    results = {'model_name' : best_result}

Типичные параметры для оптимизации градиентного бустинга: 
* число итераций алгоритма
* глубина деревьев
* скорость обучения (`learning_rate`). 
Чтобы попробовать оптимизировать модели, стоит обратиться к документации: разные реализации будут предлагать разные варианты параметров. 
   > Для классификации используем метрику `accuracy_score`

In [37]:
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20)
    }
    model = RandomForestClassifier(**params, random_state=42)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    return cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean()

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=20)
best_rf_params = study_rf.best_params

In [38]:
def objective_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3)
    }
    model = CatBoostClassifier(**params, random_seed=42, verbose=0)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    return cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean()

study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=20)
best_cat_params = study_cat.best_params

In [39]:
def objective_lgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3)
    }
    model = LGBMClassifier(**params, random_state=42)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    return cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean()

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=20)
best_lgb_params = study_lgb.best_params

[LightGBM] [Info] Number of positive: 321, number of negative: 266
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000225 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 587, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546848 -> initscore=0.187945
[LightGBM] [Info] Start training from score 0.187945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [40]:
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3)
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='logloss')
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    return cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy').mean()

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=20)
best_xgb_params = study_xgb.best_params

4. Превращаем словарь в `DataFrame` и сравниваем полученные результаты. Строим `bar_plot` для визуализации метрики качества. 

In [41]:
optuna_results = {
    "RandomForest": study_rf.best_value,
    "CatBoost": study_cat.best_value,
    "LightGBM": study_lgb.best_value,
    "XGBoost": study_xgb.best_value
}

optuna_df = pd.DataFrame.from_dict(optuna_results, orient='index', columns=['accuracy'])
print("\nРезультаты после оптимизации")
print(optuna_df.sort_values('accuracy', ascending=False))


Результаты после оптимизации
              accuracy
RandomForest  0.877355
CatBoost      0.873302
LightGBM      0.866490
XGBoost       0.863741


5. Выберете лучшую модель и попробуйте задеплоить ее в Streamlit.

* Создайте просто интерфейс для пользователя, куда бы он мог ввести необходимые данные, а вы бы ему вернули предсказание

In [42]:
optimized_models = [
    ('rf', RandomForestClassifier(**best_rf_params, random_state=42)),
    ('cat', CatBoostClassifier(**best_cat_params, random_seed=42, verbose=0)),
    ('lgb', LGBMClassifier(**best_lgb_params, random_state=42)),
    ('xgb', XGBClassifier(**best_xgb_params, random_state=42))
]

optimized_voting = VotingClassifier(estimators=optimized_models, voting='soft')
optimized_pipe = Pipeline([('preprocessor', preprocessor), ('classifier', optimized_voting)])
optimized_pipe.fit(X_train, y_train)
opt_voting_acc = accuracy_score(y_valid, optimized_pipe.predict(X_valid))
print(f"Optimized Voting Accuracy: {opt_voting_acc:.4f}")

final_results = {
    'Baseline Models': ensemble_df['accuracy'].to_dict(),
    'Optimized Models': optuna_results,
    'Voting': {'Basic': voting_acc, 'Optimized': opt_voting_acc}
}

best_model = CatBoostClassifier(**best_cat_params, random_seed=42, verbose=0)
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])
best_pipeline.fit(X_train, y_train)
joblib.dump(best_pipeline, 'best_model.pkl')

print("\nЛучшая модель:", best_model.__class__.__name__)
print("Accuracy:", accuracy_score(y_valid, best_pipeline.predict(X_valid)))

[LightGBM] [Info] Number of positive: 401, number of negative: 333
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 375
[LightGBM] [Info] Number of data points in the train set: 734, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546322 -> initscore=0.185819
[LightGBM] [Info] Start training from score 0.185819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[